<a href="https://colab.research.google.com/github/thz2711/mymachine/blob/main/anime-webui-colab/notebooks/provide_your_own_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>🌟 <code>PYOM* WebUI Colab</code></h2>

<i>*<b>P</b>rovide <b>Y</b>our <b>O</b>wn <b>M</b>odels </i> 😊

### 🤔 How to use?

1. Head over to **[⏬ `Download models` cell](#scrollTo=knRBu_TsqGb9)**

  a. Insert your **direct download link** to the `model_link` textbox

2. Upon the download finishes, you can now proceed to **[`🚀 Launch web UI` cell](#scrollTo=_cRmoHmYPcoD)**

 a. Before clicking on play, paste the **file name** of the model (and/or VAE) from the previous cell to `model` and/or `vae` textboxes

3. Generate! 😊

If things breaks, call my attention by opening an issue at this [Github Repository](https://github.com/NUROISEA/anime-webui-colab)

----

[![colab count][colab-count]](https://github.com/NUROISEA/anime-webui-colab#-notebooks)
[![last update][last-update]](https://github.com/NUROISEA/anime-webui-colab/commits/main)

[colab-count]: https://img.shields.io/github/directory-file-count/NUROISEA/anime-webui-colab/notebooks?label=other%20colabs&logo=google-colab&style=flat-square&type=file
[last-update]: https://img.shields.io/github/last-commit/NUROISEA/anime-webui-colab?label=last%20update&logo=github&style=flat-square&type=file

In [ ]:
!pip uninstall -y pillow

In [ ]:
!pip install pillow==9.5.0

<h2>📚 <code>Model resources</code></h2>

<h3>Models</h3>

<details><summary> Click to expand</summary>

NoCrypt's model listing is a good list
 - https://rentry.org/ncpt_fav_models

Direct model links listing of the models I have gathered [in my repo](https://github.com/NUROISEA/anime-webui-colab)
 - https://rentry.org/anime-webui-colab-link-dump

✨ A bunch of models that I gathered:
 - https://nuroisea.github.io/list/

</details>

<h3>VAEs</h3>

<details><summary> Click to expand</summary>

There is only 2 to pick from

 - Anything VAE
    - https://huggingface.co/NUROISEA/av3-mirror/resolve/main/anything.vae.pt
 - Waifu Diffusion VAE
    - https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt


</details>

In [ ]:
#@title ⏬ `Download models`

#@markdown You can run this multiple times

#@markdown Full compatibility with HuggingFace, Civitai somewhat supported.

#@markdown Multiple links supported, separate with commas

#@markdown Example: `link1,link2,link3`

#@markdown <small> Direct model link
model_link = "https://huggingface.co/Ntsk-1/mainNtsk/resolve/main/uaiso.safetensors" #@param { type: 'string' }

#@markdown <small> Civitai API key to download models (<b>Required</b> for Civitai links)
civitai_api = "" #@param { type: 'string' }

#@markdown <small> (Optional) Direct VAE link
vae_link = "" #@param { type: 'string' }

model_download_folder = "/content/models"
vae_download_folder = "/content/VAE"

git_flags = "-q"
wget_flags = "-q --show-progress"

try:
  utility
except NameError:
  !wget -q https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/utility.py -P /content/
  import utility

  has_downloaded_at_least_once = False

def model_download(link, folder, type='model'):
  if link == '':
    print(f'🤷‍♂️ No {type} link entered. Downloading no {type}s.')
    return

  if 'huggingface.co' in link:
    link = link.replace('?download=true', '')
    link = link.replace('/raw/', '/resolve/')
    link = link.replace('/blob/', '/resolve/')
    file_name = link.split('/')[-1]
    !{utility.aria2_download(link, folder, file_name)}

  elif 'civitai.com' in link:
    if not civitai_api:
      print(f'❌ API key is required on downloding from Civitai!')
      print(f'🤷‍♂️ Downloading nothing. Skipping {link}')
      return
    if '/api/download/' not in link:
      print(f'🛑 Please provide a valid civitai download link! Your link must contain /api/download in it!')
      print(f'👁️ See this image for reference: https://iili.io/JgVHlet.png')
      print(f'🤷‍♂️ Downloading nothing. Skipping {link}')
      return
    print(f'⏬ Downloading with wget (temp): {link}')
    civitai_link_with_token = f'{link}?token={civitai_api}'
    !{utility.wget_download(civitai_link_with_token, folder)}

  else:
    print('😕 The link you provided is not supported yet, but still downloading!')
    print('😣 Your downloaded file might not be what you expect.')
    !{utility.wget_download(link, folder)}

# Model download
for link in model_link.split(','):
  model_download(link.strip(), model_download_folder, type='model')

# VAE download
for link in vae_link.split(','):
  model_download(link.strip(), vae_download_folder, type='VAE')

print("\n📃 List of models downloaded:")
!ls -1 {model_download_folder}

print("\n📃 List of VAEs downloaded:")
!ls -1 {vae_download_folder}

has_downloaded_at_least_once = True

In [ ]:
#@title 🚀 `Launch web UI`

#@markdown ### ⭐ Model selection
#@markdown  It could take 4-7 minutes to see a link to the web UI, please be patient! :) </small>

#@markdown <small> Select a model to forcefully load
model = "uaiso.safetensors" #@param { type: 'string' }
#@markdown <small> Select a VAE to forcefully load
vae = "" #@param { type: 'string' }

#@markdown <small> Select ControlNet models to use [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Selecting-ControlNet-models) </small>
controlnet = "none" #@param ['none','v1.0','v1.0-diff','t2i','v1.1']
#@markdown ### 🔧 Web UI settings
#@markdown <small> This option only affects the **first** launch [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/First-launch-web-UI-settings) </small>
webui_version = "latest" #@param ["fallback","stable","latest","latest-dev","ui-redesign"]
extensions_version = "experimental" #@param ["none","lite","stable","latest","experimental"]
#@markdown <small> This saves your generation to Google Drive [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Saving-outputs-to-Google-Drive) </small>
outputs_to_drive = False #@param {type:'boolean'}
output_drive_folder = "AI/Generated" #@param {type:'string'}
#@markdown <small> Change tunnels if you have connection issues with `gradio` [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Using-different-tunnels) </small>
tunnel = "gradio" #@param ['gradio', 'ngrok', 'cloudflared', 'localhostrun', 'remotemoe']
ngrok_token = "" #@param { type:'string' }
ngrok_region = "auto" #@param ["auto", "ap", "au", "eu", "in", "jp", "sa", "us"]
#@markdown <small> Override the default launch arguments [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Overriding-default-launch-arguments)
default_arguments_override = "" #@param { type:'string' }

try:
  has_downloaded_at_least_once
except:
  sys.exit("😣 You haven't downloaded anything, please run the previous cell first!")

ui_config_dictionary = {
  "txt2img/Prompt/value": "masterpiece, best quality, ",
  "txt2img/Negative prompt/value": "(worst quality, low quality:1.4)"
}

config_dictionary = {
  "quicksettings_list": [
    "sd_model_checkpoint",
    "sd_vae",
    "CLIP_stop_at_last_layers",
    "token_merging_ratio",
  ],
}

if not utility.has_run:
  utility.log_usage("notebook-provide_your_own_models")

  !{utility.install_webui(webui_version)}

  for extension_link in utility.extensions_list(extensions_version,webui_version,controlnet):
    extension_name = extension_link.split('/')[-1]
    print(f'  └ {extension_name}')
    !git clone {git_flags} {extension_link}
  # !git clone {git_flags} github.com/username/<repo> {extensions_folder}/<repo>

  for config_file in utility.configs_list():
    !wget -q {config_file} -P {utility.web_ui_folder}/

  for embedding in utility.embeddings_list():
    !wget -q {embedding} -P {utility.embeddings_folder}/
  # !wget {wget_flags} site.com/embedding.pt -P {utility.embeddings_folder}/

  utility.dictionary_to_json(f"{utility.web_ui_folder}/ui-config.json", ui_config_dictionary)
  utility.dictionary_to_json(f"{utility.web_ui_folder}/config.json", config_dictionary)

  for patch in utility.patch_list():
    !{patch}
  %env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

  utility.has_run = True

utility.output_to_gdrive(outputs_to_drive, output_drive_folder)

for controlnet_model_link in utility.controlnet_list(controlnet,webui_version,extensions_version):
  !{utility.download_controlnet(controlnet_model_link)}

%cd {utility.web_ui_folder}

arg_list = utility.arguments(
  model=f"{model_download_folder}/{model}" if model else "",
  vae=f"{vae_download_folder}/{vae}" if vae else "",
  tunnel=tunnel,
  ng_token=ngrok_token,
  ng_region=ngrok_region,
  default_override=default_arguments_override,
  extra_args=f"--ckpt-dir {model_download_folder} --vae-dir {vae_download_folder}",
)

args = " ".join(arg_list)
!python launch.py {args}
#@markdown <i><small> Did something break? Please report it on [Github](https://github.com/NUROISEA/anime-webui-colab/issues) or fill out this [Google Form](https://forms.gle/dnaJojPR4yqvr9dN8) (no sign-in needed)!

In [ ]:
#@title 📱 `Colab Mobile Keep Alive`
#@markdown <small> Run this cell (before running 🚀 `Launch web UI`) to keep the tab alive in mobile [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Keeping-colab-tab-alive-in-mobile) </small>

%%html
<b>Press play on the music player to keep the tab alive before running the start cell (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title 💾 `Saving images`

#@markdown <small>The zip file will be visible at the files tab.</small>
from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = f"outputs-{str_date}.zip"

%cd {utility.web_ui_folder}
print("Zipping...")
!zip -qr /content/{archive_name} outputs
print(f"\033[92mZipped. You can now find {archive_name} at the files tab.\033[0m")

# ----

#@markdown <small>This copies the zip file to your Google Drive</small>
copy_to_gdrive = False #@param {type:'boolean'}
gdrive_folder = "AI/Generated" #@param { 'type': 'string' }

if copy_to_gdrive:
  utility.log_usage('zip-to-gdrive')
  from google.colab import drive

  print("Mounting to Google Drive...")
  drive.mount('/content/drive')
  if gdrive_folder == "":
    gdrive_folder = "AI/Generated"

  drive_folder = f"/content/drive/MyDrive/{gdrive_folder}"

  !mkdir -p {drive_folder}
  !cp /content/{archive_name} {drive_folder}
  print(f"\033[92mCopied to {gdrive_folder}!\033[0m")
#@markdown <small>Upload the zip file to a temporary file hosting [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Uploading-to-temporary-file-hosts)</small>
upload_to_temp = True #@param {type:'boolean'}
temp_file_host = "bashupload.com" #@param ["bashupload.com", "keep.sh", "temp.sh", "transfer.sh"]

host_dictionary = {
  "bashupload.com": "https://bashupload.com",
  "keep.sh": "https://free.keep.sh",
  "temp.sh": "https://temp.sh",
  "transfer.sh": f"https://transfer.sh/{archive_name}",
}
host_link = host_dictionary[temp_file_host] # not sure if colab can do this directly at {}

if upload_to_temp:
  utility.log_usage('zip-to-temp-host')
  utility.log_usage(f'zip-temp-host-{temp_file_host}')
  print(f"Uploading to {temp_file_host}...")
  !curl -T /content/{archive_name} {host_link}
  print("\n\033[92m^^^ This is your download link! ^^^\033[0m")
#@markdown <small>Delete colab outputs (the saved zip is unaffected)</small>
delete_local_files = False #@param {type:'boolean'}
if delete_local_files:
  utility.log_usage('local-delete-outputs')
  %cd {utility.web_ui_folder}
  !rm -rf outputs
  print("\033[91mDeleted the outputs folder.\033[0m")